In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [30]:
train=pd.read_csv('labeledTrainData.tsv/labeledTrainData.tsv',delimiter='\t',quoting=3)
test=pd.read_csv('testData.tsv/testData.tsv',delimiter='\t',quoting=3)

In [31]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [34]:
from bs4 import BeautifulSoup
import re
import nltk
nltk.download('stopwords') #nltk.download() will show you all the package to download.
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from nltk.corpus import stopwords
clean_review=[]
for i in range(25000):
    
    review=train['review'][i]
    remove_markup=BeautifulSoup(review)
    remove_markup=remove_markup.get_text()
    
    letter_only=re.sub('[^a-zA-Z]',' ',remove_markup)
    letter_only=letter_only.lower()
    words=letter_only.split()

    words=[ps.stem(x) for x in words if x not in set(stopwords.words('english'))]
    review=' '.join(words)
    clean_review.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sKuma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)
x=cv.fit_transform(clean_review).toarray()
y=train.iloc[:,1].values

In [41]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier(n_estimators=100)
classifier.fit(x,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [42]:
clean_review_test=[]
for i in range(25000):
    
    review=test.iloc[i,1]
    remove_markup=BeautifulSoup(review)
    remove_markup=remove_markup.get_text()
    
    letter_only=re.sub('[^a-zA-Z]',' ',remove_markup)
    letter_only=letter_only.lower()
    words=letter_only.split()

    words=[ps.stem(x) for x in words if x not in set(stopwords.words('english'))]
    review=' '.join(words)
    clean_review_test.append(review)

In [43]:
test_set=cv.fit_transform(clean_review_test).toarray()
pred_y=classifier.predict(test_set)

In [45]:
submission=pd.DataFrame({'id':test['id'],'sentiment':pred_y})
submission.to_csv('Bag_of_words.csv',index=False,quoting=3)